# Data Cleaning

In [10]:
import time
import utils
import requests
from pathlib import Path

import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt

In [11]:
file1 = 'data/plant_pollinator_diversity_set1.csv'
file2 = 'data/plant_pollinator_diversity_set2.csv'

In [12]:
df1 = pd.read_csv(file1)
df2 = pd.read_csv(file2)
df2.head()

,DBCODE,ENTITY,COMPLEX,MEADOW,PLOT_ID,YEAR,SAMPLEDATE,WATCH,OBSERVER,PLOT,FLW_STATUS,PLTSP_CODE,PLTSP_NAME,NO_STALK,NO_FLWS,QC_NOTES
0,SA026,2,Bunchgrass,BD,BGD01,2011,2011-07-20,1.0,ND,1,NaN,NaN,NaN,NaN,NaN,NaN
1,SA026,2,Bunchgrass,BD,BGD01,2011,2011-07-25,2.0,Andy,1,FLW,ACHIMILL,Achillea millefolium,4.0,36.0,NaN
2,SA026,2,Bunchgrass,BD,BGD01,2011,2011-07-25,2.0,Andy,1,FLW,ERIGFOLI,Erigeron foliosus,14.0,1.0,NaN
3,SA026,2,Bunchgrass,BD,BGD01,2011,2011-07-25,2.0,Andy,1,FLW,VICIAMER,Vicia americana,3.0,2.3,NaN
4,SA026,2,Bunchgrass,BD,BGD01,2011,2011-08-02,3.0,"Andy, Tim",1,FLW,ACHIMILL,Achillea millefolium,2.0,26.0,NaN


In [13]:
df1.head()

,DBCODE,ENTITY,COMPLEX,MEADOW,PLOT_ID,YEAR,SAMPLEDATE,WATCH,OBSERVER,PLOT,...,PPI_STATUS,NO_INT,PLTSP_CODE,PLTSP_NAME,VISSP_CODE,VISSP_NAME,VISSP_TYPE,REF_NO,VISSP_NO,QC_NOTES
0,SA026,1,Bunchgrass,BD,BGD01,2011,2011-07-24,2.0,NaN,1,...,1,2.0,ACHIMILL,Achillea millefolium,EPICPUNC,Epicauta puncticollis,predaceous beetle,NaN,1,NaN
1,SA026,1,Bunchgrass,BD,BGD01,2011,2011-07-24,2.0,NaN,1,...,1,1.0,ACHIMILL,Achillea millefolium,EVYLS5M,Evylaeus sp 5M,halictine bee,NaN,1,NaN
2,SA026,1,Bunchgrass,BD,BGD01,2011,2011-07-24,2.0,NaN,1,...,1,2.0,ERIGFOLI,Erigeron foliosus,MUSCGEN3,Muscoid genus 3,herbivorous muscoid fly,NaN,1,NaN
3,SA026,1,Bunchgrass,BD,BGD01,2011,2011-07-24,2.0,NaN,1,...,1,2.0,ERIGFOLI,Erigeron foliosus,MUSCGEN3,Muscoid genus 3,herbivorous muscoid fly,NaN,2,NaN
4,SA026,1,Bunchgrass,BD,BGD01,2011,2011-07-24,2.0,NaN,1,...,1,2.0,ERIGFOLI,Erigeron foliosus,TRICORNA,Trichodes ornatus,bee-parasitic beetle,NaN,1,NaN


In [14]:
# Drop unnecessary/redundant columns for our data analysis

def drop_columns(dataframe, todrop):
    # dataframe: Pandas Dataset to drop columns from
    # todrop: list of columns to drop
    df = dataframe.drop(todrop, axis=1)
    return df

In [15]:
drop_cols_1 = ['DBCODE', 'ENTITY', 'COMPLEX', 'MEADOW', 'START_TIME', 'END_TIME', 'MINUTE',  'YEAR', 'OBSERVER', 'PLTSP_NAME', 'VISSP_NAME', 'QC_NOTES']
drop_cols_2 = ['DBCODE', 'ENTITY', 'COMPLEX', 'MEADOW', 'YEAR', 'OBSERVER', 'PLTSP_NAME', 'QC_NOTES']

df1 = drop_columns(df1, drop_cols_1)
df2 = drop_columns(df2, drop_cols_2)
df1.head()

,PLOT_ID,SAMPLEDATE,WATCH,PLOT,CLOUDS,WIND,TEMP,PPI_STATUS,NO_INT,PLTSP_CODE,VISSP_CODE,VISSP_TYPE,REF_NO,VISSP_NO
0,BGD01,2011-07-24,2.0,1,S,ST,NaN,1,2.0,ACHIMILL,EPICPUNC,predaceous beetle,NaN,1
1,BGD01,2011-07-24,2.0,1,S,ST,NaN,1,1.0,ACHIMILL,EVYLS5M,halictine bee,NaN,1
2,BGD01,2011-07-24,2.0,1,S,ST,NaN,1,2.0,ERIGFOLI,MUSCGEN3,herbivorous muscoid fly,NaN,1
3,BGD01,2011-07-24,2.0,1,S,ST,NaN,1,2.0,ERIGFOLI,MUSCGEN3,herbivorous muscoid fly,NaN,2
4,BGD01,2011-07-24,2.0,1,S,ST,NaN,1,2.0,ERIGFOLI,TRICORNA,bee-parasitic beetle,NaN,1


In [16]:
# Merge datasets (union on PLOT_ID)
dff = df1.merge(df2, on=['PLOT_ID', 'SAMPLEDATE', 'PLTSP_CODE', 'PLOT', 'WATCH'], how='outer')
dff.head()

,PLOT_ID,SAMPLEDATE,WATCH,PLOT,CLOUDS,WIND,TEMP,PPI_STATUS,NO_INT,PLTSP_CODE,VISSP_CODE,VISSP_TYPE,REF_NO,VISSP_NO,FLW_STATUS,NO_STALK,NO_FLWS
0,BGD01,2011-07-24,2.0,1,S,ST,NaN,1.0,2.0,ACHIMILL,EPICPUNC,predaceous beetle,NaN,1.0,NaN,NaN,NaN
1,BGD01,2011-07-24,2.0,1,S,ST,NaN,1.0,1.0,ACHIMILL,EVYLS5M,halictine bee,NaN,1.0,NaN,NaN,NaN
2,BGD01,2011-07-24,2.0,1,S,ST,NaN,1.0,1.0,ACHIMILL,ANTHASP1,herbivorous beetle,NaN,1.0,NaN,NaN,NaN
3,BGD01,2011-07-24,2.0,1,S,ST,NaN,1.0,1.0,ACHIMILL,ERISHIRT,decomposer hover-fly,NaN,1.0,NaN,NaN,NaN
4,BGD01,2011-07-24,2.0,1,S,ST,NaN,1.0,2.0,ERIGFOLI,MUSCGEN3,herbivorous muscoid fly,NaN,1.0,NaN,NaN,NaN


In [17]:
# Drop NaNs
dff = dff.dropna(subset=['TEMP', 'FLW_STATUS', 'NO_STALK', 'NO_FLWS'])
dff.sample(15)


,PLOT_ID,SAMPLEDATE,WATCH,PLOT,CLOUDS,WIND,TEMP,PPI_STATUS,NO_INT,PLTSP_CODE,VISSP_CODE,VISSP_TYPE,REF_NO,VISSP_NO,FLW_STATUS,NO_STALK,NO_FLWS
12234,CPM01,2017-06-20,1.0,1,S,BR,23.0,1.0,1.0,LOMAMART,SYRPSPLB,NaN,NaN,1.0,FLW,221.0,4.4
53982,LOO06,2017-07-06,2.0,6,S,ST,26.0,1.0,1.0,DRYMGLAN,MUSCGEN1,NaN,NaN,1.0,FLW,7.0,1.3
73412,RP209,2017-07-19,4.0,9,S,BR,22.0,1.0,1.0,ERIOLANA,EPICPUNC,NaN,NaN,1.0,FLW,23.0,3.0
34708,FM209,2018-07-12,3.0,9,S,BR,22.0,1.0,3.0,ORTHIMBR,BOMBBIFA,NaN,NaN,1.0,FLW,59.0,3.0
42224,LOB03,2016-07-19,4.0,3,S,ST,24.0,1.0,4.0,SENETRIA,APISMELL,social bee,no number,1.0,FLW,148.0,2.1
48849,LOM07,2017-07-25,4.0,7,S,ST,24.0,1.0,1.0,ACHIMILL,MUSCGEN1,NaN,NaN,1.0,FLW,6.0,5.3
47937,LOM06,2015-07-14,3.0,6,C,BR,19.0,1.0,7.0,ACHIMILL,ESCHCONS,herbivorous beetle,35507,2.0,FLW,6.0,6.3
47186,LOM04,2018-08-07,5.0,4,S,BR,26.0,1.0,3.0,PERIGAIR,APISMELL,NaN,NaN,1.0,FLW,10.0,6.2
51170,LOO01,2017-07-13,3.0,1,S,ST,21.0,1.0,1.0,ERIOLANA,ESCHCONS,NaN,NaN,1.0,FLW,16.0,1.3
71343,RP204,2018-07-09,2.0,4,S,BR,22.0,1.0,3.0,ACMINEVA,BOMBVOSN,NaN,NaN,1.0,FLW,40.0,4.0


In [18]:
dff.describe()

,WATCH,PLOT,TEMP,PPI_STATUS,NO_INT,VISSP_NO,NO_STALK,NO_FLWS
count,24896.000000,24896.000000,24896.000000,24896.0,24883.000000,24896.000000,24896.000000,24896.000000
mean,2.829531,5.373353,22.853109,1.0,3.230278,1.762010,50.104354,8.037982
std,1.278187,2.899112,4.419978,0.0,4.027019,1.794089,59.315180,18.698536
min,1.000000,1.000000,10.000000,1.0,1.000000,1.000000,1.000000,0.000000
25%,2.000000,3.000000,20.000000,1.0,1.000000,1.000000,9.000000,1.500000
50%,3.000000,5.000000,23.000000,1.0,2.000000,1.000000,29.000000,2.600000
75%,4.000000,8.000000,26.000000,1.0,4.000000,2.000000,75.000000,7.000000
max,7.000000,10.000000,37.000000,1.0,200.000000,26.000000,781.000000,380.000000
